## Import modules

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns 
import pandas as pd
import sqlite3
import scipy.stats as scipy

from sklearn.metrics import accuracy_score

from numpy import random
from IPython.display import display


### Connect to sqlite database

In [3]:
with sqlite3.connect('./data/database.sqlite') as con :
    countries = pd.read_sql_query("SELECT * from Country",con)
    leagues = pd.read_sql_query("SELECT * from League",con)
    matches = pd.read_sql_query("SELECT * from Match",con)
    players = pd.read_sql_query("SELECT * from Player",con)
    players_attributes = pd.read_sql_query("SELECT * from Player_Attributes",con)
    teams = pd.read_sql_query("SELECT * from Team",con)
    teams_attributes = pd.read_sql_query("SELECT * from Team_Attributes",con)

In [9]:
print(countries.columns.values)
print(leagues.columns.values)
print(matches.columns.values)
print(players.columns.values)
print(players_attributes.columns.values)
print(teams.columns.values)
print(teams_attributes.columns.values)

['id' 'name']
['id' 'country_id' 'name']
['id' 'country_id' 'league_id' 'season' 'stage' 'date' 'match_api_id'
 'home_team_api_id' 'away_team_api_id' 'home_team_goal' 'away_team_goal'
 'home_player_X1' 'home_player_X2' 'home_player_X3' 'home_player_X4'
 'home_player_X5' 'home_player_X6' 'home_player_X7' 'home_player_X8'
 'home_player_X9' 'home_player_X10' 'home_player_X11' 'away_player_X1'
 'away_player_X2' 'away_player_X3' 'away_player_X4' 'away_player_X5'
 'away_player_X6' 'away_player_X7' 'away_player_X8' 'away_player_X9'
 'away_player_X10' 'away_player_X11' 'home_player_Y1' 'home_player_Y2'
 'home_player_Y3' 'home_player_Y4' 'home_player_Y5' 'home_player_Y6'
 'home_player_Y7' 'home_player_Y8' 'home_player_Y9' 'home_player_Y10'
 'home_player_Y11' 'away_player_Y1' 'away_player_Y2' 'away_player_Y3'
 'away_player_Y4' 'away_player_Y5' 'away_player_Y6' 'away_player_Y7'
 'away_player_Y8' 'away_player_Y9' 'away_player_Y10' 'away_player_Y11'
 'home_player_1' 'home_player_2' 'home_player_3' 

In [11]:
data1 = pd.read_csv("./data/EPL/19_20.csv")

In [12]:
data1.columns.values

array(['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
       'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST',
       'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D',
       'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD',
       'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD',
       'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5',
       'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh',
       'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH',
       'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA',
       'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD',
       'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH',
       'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5',
       'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH',
       'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'Av

In [34]:
col = ["Div","Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","B365H","B365D",
      "B365A","BWH","BWD","BWA","IWH","IWD","IWA","PSH","PSD","PSA","WHH","WHD","WHA",
      "VCH","VCD","VCA"]

data = data1[col]

In [35]:
data.shape

(380, 25)

In [36]:
data2 = pd.read_csv("./data/EPL/18_19.csv")
data2 = data2[col]
print(data2.columns.values)

['Div' 'Date' 'HomeTeam' 'AwayTeam' 'FTHG' 'FTAG' 'FTR' 'B365H' 'B365D'
 'B365A' 'BWH' 'BWD' 'BWA' 'IWH' 'IWD' 'IWA' 'PSH' 'PSD' 'PSA' 'WHH' 'WHD'
 'WHA' 'VCH' 'VCD' 'VCA']


In [37]:
data2["Div"]

0      E0
1      E0
2      E0
3      E0
4      E0
       ..
375    E0
376    E0
377    E0
378    E0
379    E0
Name: Div, Length: 380, dtype: object

In [38]:
data2["Date"].factorize()[0] # 0 : codes, 1 : uniques

array([  0,   1,   1,   1,   1,   1,   1,   2,   2,   2,   3,   3,   3,
         3,   3,   3,   4,   4,   4,   5,   6,   6,   6,   6,   6,   6,
         7,   7,   7,   8,   9,   9,   9,   9,   9,   9,   9,  10,  10,
        10,  11,  11,  11,  11,  11,  11,  11,  12,  12,  13,  14,  14,
        14,  14,  14,  14,  14,  14,  15,  15,  16,  16,  16,  16,  16,
        16,  16,  16,  17,  18,  19,  20,  20,  20,  20,  20,  20,  21,
        21,  21,  22,  22,  22,  22,  22,  22,  22,  22,  23,  24,  25,
        25,  25,  25,  25,  25,  26,  26,  26,  27,  28,  28,  28,  28,
        28,  28,  28,  29,  29,  30,  31,  31,  31,  31,  31,  31,  32,
        32,  32,  32,  33,  33,  33,  33,  33,  33,  33,  34,  34,  35,
        36,  37,  37,  37,  37,  37,  37,  38,  38,  38,  39,  39,  39,
        39,  40,  40,  40,  40,  40,  40,  41,  41,  41,  41,  41,  41,
        41,  41,  42,  43,  44,  44,  44,  44,  44,  44,  44,  45,  45,
        45,  46,  47,  47,  47,  47,  47,  47,  47,  47,  48,  4

In [65]:
codes, uniques = data2["HomeTeam"].factorize()

In [69]:
print(uniques)

Index(['Man United', 'Bournemouth', 'Fulham', 'Huddersfield', 'Newcastle',
       'Watford', 'Wolves', 'Arsenal', 'Liverpool', 'Southampton', 'Cardiff',
       'Chelsea', 'Everton', 'Leicester', 'Tottenham', 'West Ham', 'Brighton',
       'Burnley', 'Man City', 'Crystal Palace'],
      dtype='object')


In [68]:
dic = {}
for i in range(len(uniques)):
    dic[uniques[i]] = codes[i]
print(dic)

{'Man United': 0, 'Bournemouth': 1, 'Fulham': 2, 'Huddersfield': 3, 'Newcastle': 4, 'Watford': 5, 'Wolves': 6, 'Arsenal': 7, 'Liverpool': 8, 'Southampton': 9, 'Cardiff': 10, 'Chelsea': 11, 'Everton': 12, 'Leicester': 13, 'Tottenham': 14, 'West Ham': 15, 'Brighton': 16, 'Burnley': 17, 'Man City': 18, 'Crystal Palace': 19}


In [39]:
v = data2["Date"]

In [54]:
v[0]


'10/08/2018'

In [57]:
from datetime import datetime

In [63]:
dt = datetime.strptime(v[0],"%d/%m/%Y")

In [64]:
dt

datetime.datetime(2018, 8, 10, 0, 0)